### 1.KNN无线定位方法

数据集来自<a>https://github.com/jiangqideng/codeInBlogs/blob/master/main_loc_test.ipynb  坐标单位为cm

In [1]:
# 读取mat数据集
import numpy as np
import scipy.io as scio
offline_data = scio.loadmat('offline_data_random.mat')
online_data = scio.loadmat('online_data.mat')
location_ori, finger_ori = offline_data['offline_location'], offline_data['offline_rss']
location_val, finger_val = online_data['trace'][0:100, :], online_data['rss'][0:100, :]
del offline_data
del online_data

In [2]:
class knn_location:
    def __init__(self,finger_ori,location_ori,finger_val,location_val):
        self.finger_ori=finger_ori
        self.location_ori=location_ori
        self.finger_val=finger_val
        self.location_val=location_val

    def compute(self):
        d_all=[]
        for j in range(0,len(finger_val)):
            for i in range(0,len(self.finger_ori)):
                # (np.sum((np.array([5,4])-np.array([1,1]))**2))**0.5 计算欧氏距离
                d=(np.sum((finger_val[j]-finger_ori[i])**2))**0.5
                d_all.append(d)
        return d_all

In [3]:
# 平均定位误差
def accuracy(pre, location_val):
    return np.mean(np.sqrt(np.sum((pre - location_val)**2, 1)))

# 冒泡k次找出k个最接近的坐标
def bubble(d_all,label,k): 
  length = len(d_all) 
  for i in range(k): 
    for j in range(i+1,length): 
      if d_all[i] > d_all[j]: 
        d_all[j],d_all[i] = d_all[i],d_all[j]
        label[j],label[i]= label[i],label[j]
  return d_all[0:k],label[0:k]

In [4]:
knn_regressor=knn_location(finger_ori,location_ori,finger_val,location_val)
d_all=knn_regressor.compute()

In [5]:
# 把所有的欧式距离转换为np数组
d_all=np.array(d_all)
# 重组为每一行表示一个待测点与指纹集的所有距离
d_all=d_all.reshape(-1,len(finger_ori))

In [6]:
k=40
d_sort_all=[]
label_sort_all=[]

for val in range(0,len(finger_val)):
    label=np.array(range(0,len(finger_ori)))
    d_sort,label_sort=bubble(d_all[val],label,k)
    d_sort_all.append(d_sort)
    label_sort_all.append(label_sort)

In [7]:
pre=[]
for d in range(0,len(d_sort_all)):
    k_location=location_ori[label_sort_all[d]]
    pre.append(np.sum(k_location.T,axis=1)/k)

In [8]:
pre=np.array(pre)
# 计算平均定位误差
errors=accuracy(pre,location_val)
print("平均定位误差",errors/100,"m")

平均定位误差 2.300836741056577 m
